<a href="https://colab.research.google.com/github/ReddyJaiSaiPreetham/5CNETWORK/blob/main/5C_Network_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
pip install tensorflow keras opencv-python numpy matplotlib pydicom scikit-image fastapi streamlit uvicorn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [15]:
import requests
import zipfile
import os

# URL to the dataset
url = 'https://dicom5c.blob.core.windows.net/public/Data.zip'
local_zip = 'Data.zip'

# Download the file from the URL
print("Downloading dataset...")
response = requests.get(url)

# Save the content to a local file
with open(local_zip, 'wb') as f:
    f.write(response.content)

print("Download complete. Extracting files...")

# Extract the ZIP file
with zipfile.ZipFile(local_zip, 'r') as zip_ref:
    zip_ref.extractall('data')

print("Extraction complete. Dataset ready for use!")

# Optionally, remove the zip file after extraction
os.remove(local_zip)


Download complete. Extracting files...
Extraction complete. Dataset ready for use!


In [19]:
import os
import cv2

# Define paths
dataset_path = '/content/drive/MyDrive/Data/Data'  # Set your dataset folder here

# Initialize lists to store image data and corresponding masks
images = []
masks = []
image_files = []
mask_files = []

# Loop through files in the dataset folder
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(".tif"):
            file_path = os.path.join(root, file)

            # Check if it's a mask or normal image
            if "_mask" in file:
                mask_image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Load mask as grayscale
                if mask_image is not None:
                    masks.append(mask_image)
                    mask_files.append(file)
            else:
                normal_image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)  # Load normal image
                if normal_image is not None:
                    images.append(normal_image)
                    image_files.append(file)

# Check for mismatches in file names
image_base_names = [f.split(".")[0] for f in image_files]
mask_base_names = [f.split("_mask")[0] for f in mask_files]

# Find unmatched images
unmatched_images = list(set(image_base_names) - set(mask_base_names))
unmatched_masks = list(set(mask_base_names) - set(image_base_names))

if unmatched_images:
    print(f"Unmatched images: {unmatched_images}")
if unmatched_masks:
    print(f"Unmatched masks: {unmatched_masks}")

# Remove unmatched images/masks
if unmatched_images:
    for img in unmatched_images:
        idx = image_base_names.index(img)
        del images[idx]

if unmatched_masks:
    for mask in unmatched_masks:
        idx = mask_base_names.index(mask)
        del masks[idx]

# Check if the mismatch is resolved
if len(images) == len(masks):
    print("Mismatch resolved. Proceeding with train-test split.")
else:
    print(f"Still a mismatch: {len(images)} images and {len(masks)} masks.")

# Convert lists to numpy arrays
images = np.array(images)
masks = np.array(masks)

# Split the dataset into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.2, random_state=42)

# Print out the sizes of the training and testing sets
print(f"Training set size: {X_train.shape[0]} images, Test set size: {X_test.shape[0]} images")


Unmatched images: ['TCGA_HT_7605_19950916_2']
Mismatch resolved. Proceeding with train-test split.
Training set size: 740 images, Test set size: 185 images


In [6]:
import os
from PIL import Image
import numpy as np

# Define the base directory
base_dir = "/content/drive/MyDrive/Data/Data"

# Lists to store images and masks
images = []
masks = []

# Loop through folders in base_dir
for folder_name in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder_name)

    if os.path.isdir(folder_path):
        print(f"Processing folder: {folder_name}")

        # Loop through files in each folder
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)

            # Check for image and mask files (tif format)
            if file_name.endswith(('.tif', '.tiff')):
                if 'mask' in file_name.lower():
                    print(f"Loading mask: {file_name}")
                    mask = Image.open(file_path)
                    masks.append(np.array(mask))
                else:
                    print(f"Loading image: {file_name}")
                    image = Image.open(file_path)
                    images.append(np.array(image))

# Check if any images and masks were loaded
if len(images) == 0 or len(masks) == 0:
    raise ValueError("No images or masks loaded. Please check the dataset.")

print(f"Loaded {len(images)} images and {len(masks)} masks.")


Processing folder: TCGA_HT_7860_19960513
Loading image: TCGA_HT_7860_19960513_21.tif
Loading image: TCGA_HT_7860_19960513_5.tif
Loading image: TCGA_HT_7860_19960513_4.tif
Loading image: TCGA_HT_7860_19960513_6.tif
Loading image: TCGA_HT_7860_19960513_7.tif
Loading mask: TCGA_HT_7860_19960513_7_mask.tif
Loading image: TCGA_HT_7860_19960513_9.tif
Loading image: TCGA_HT_7860_19960513_8.tif
Loading mask: TCGA_HT_7860_19960513_6_mask.tif
Loading mask: TCGA_HT_7860_19960513_8_mask.tif
Loading mask: TCGA_HT_7860_19960513_9_mask.tif
Loading mask: TCGA_HT_7860_19960513_4_mask.tif
Loading image: TCGA_HT_7860_19960513_3.tif
Loading image: TCGA_HT_7860_19960513_22.tif
Loading mask: TCGA_HT_7860_19960513_22_mask.tif
Loading mask: TCGA_HT_7860_19960513_2_mask.tif
Loading mask: TCGA_HT_7860_19960513_21_mask.tif
Loading mask: TCGA_HT_7860_19960513_3_mask.tif
Loading mask: TCGA_HT_7860_19960513_5_mask.tif
Loading image: TCGA_HT_7860_19960513_14.tif
Loading mask: TCGA_HT_7860_19960513_19_mask.tif
Loadin

In [7]:
from tensorflow.keras import layers, models

# U-Net++ Model
def unet_plus_plus(input_size=(256, 256, 3)):
    inputs = layers.Input(input_size)

    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    # Decoder and additional convolution blocks for U-Net++
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)

    up1 = layers.UpSampling2D(size=(2, 2))(conv2)
    up1 = layers.concatenate([up1, conv1], axis=3)
    conv3 = layers.Conv2D(64, 3, activation='relu', padding='same')(up1)
    conv3 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv3)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv3)

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Attention U-Net Model
def attention_unet(input_size=(256, 256, 3)):
    inputs = layers.Input(input_size)

    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Attention block
    attention = layers.Conv2D(1, 1, activation='sigmoid')(conv2)

    # Decoder
    up1 = layers.UpSampling2D(size=(2, 2))(attention)
    up1 = layers.concatenate([up1, conv1], axis=3)
    conv3 = layers.Conv2D(64, 3, activation='relu', padding='same')(up1)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv3)

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [13]:
import os
from PIL import Image
import numpy as np

# Define the base directory
base_dir = "/content/drive/MyDrive/Data/Data"

# Lists to store images and masks
images = []
masks = []

# Loop through folders in base_dir
for folder_name in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder_name)

    if os.path.isdir(folder_path):
        print(f"Processing folder: {folder_name}")

        # Loop through files in each folder
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)

            # Check for image and mask files (tif format)
            if file_name.endswith(('.tif', '.tiff')):
                if 'mask' in file_name.lower():
                    print(f"Loading mask: {file_name}")
                    mask = Image.open(file_path)
                    masks.append(np.array(mask))
                else:
                    print(f"Loading image: {file_name}")
                    image = Image.open(file_path)
                    images.append(np.array(image))

# Check if any images and masks were loaded
if len(images) == 0 or len(masks) == 0:
    print("No images or masks were loaded. Please check the directory structure and file names.")
else:
    print(f"Loaded {len(images)} images and {len(masks)} masks.")


Processing folder: TCGA_HT_7860_19960513
Loading image: TCGA_HT_7860_19960513_21.tif
Loading image: TCGA_HT_7860_19960513_5.tif
Loading image: TCGA_HT_7860_19960513_4.tif
Loading image: TCGA_HT_7860_19960513_6.tif
Loading image: TCGA_HT_7860_19960513_7.tif
Loading mask: TCGA_HT_7860_19960513_7_mask.tif
Loading image: TCGA_HT_7860_19960513_9.tif
Loading image: TCGA_HT_7860_19960513_8.tif
Loading mask: TCGA_HT_7860_19960513_6_mask.tif
Loading mask: TCGA_HT_7860_19960513_8_mask.tif
Loading mask: TCGA_HT_7860_19960513_9_mask.tif
Loading mask: TCGA_HT_7860_19960513_4_mask.tif
Loading image: TCGA_HT_7860_19960513_3.tif
Loading image: TCGA_HT_7860_19960513_22.tif
Loading mask: TCGA_HT_7860_19960513_22_mask.tif
Loading mask: TCGA_HT_7860_19960513_2_mask.tif
Loading mask: TCGA_HT_7860_19960513_21_mask.tif
Loading mask: TCGA_HT_7860_19960513_3_mask.tif
Loading mask: TCGA_HT_7860_19960513_5_mask.tif
Loading image: TCGA_HT_7860_19960513_14.tif
Loading mask: TCGA_HT_7860_19960513_19_mask.tif
Loadin

In [14]:

# Define the target size for resizing
target_size = (128, 128)  # Adjust as needed

def preprocess_image_and_mask(image, mask):
    # Resize image and mask to target size
    image_resized = Image.fromarray(image).resize(target_size)
    mask_resized = Image.fromarray(mask).resize(target_size)

    # Normalize pixel values (convert to float and divide by 255)
    image_resized = np.array(image_resized) / 255.0
    mask_resized = np.array(mask_resized) / 255.0

    return image_resized, mask_resized

# Preprocess all images and masks
preprocessed_images = []
preprocessed_masks = []

for img, mask in zip(images, masks):
    pre_img, pre_mask = preprocess_image_and_mask(img, mask)
    preprocessed_images.append(pre_img)
    preprocessed_masks.append(pre_mask)

# Convert to numpy arrays
X = np.array(preprocessed_images)
y = np.array(preprocessed_masks)

print(f"Preprocessed {len(X)} images and {len(y)} masks.")


Preprocessed 925 images and 925 masks.


In [15]:

from sklearn.model_selection import train_test_split

# Split the dataset into 80% training and 20% validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]} images, Validation set size: {X_val.shape[0]} images.")


Training set size: 740 images, Validation set size: 185 images.


In [16]:
from tensorflow.keras import layers, Model

# Define the U-Net++ model
def conv_block(x, num_filters):
    x = layers.Conv2D(num_filters, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(num_filters, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

def unet_plus_plus(input_shape=(128, 128, 3), num_classes=1):
    inputs = layers.Input(input_shape)

    # Encoder
    e1 = conv_block(inputs, 64)
    p1 = layers.MaxPooling2D((2, 2))(e1)

    e2 = conv_block(p1, 128)
    p2 = layers.MaxPooling2D((2, 2))(e2)

    e3 = conv_block(p2, 256)
    p3 = layers.MaxPooling2D((2, 2))(e3)

    e4 = conv_block(p3, 512)
    p4 = layers.MaxPooling2D((2, 2))(e4)

    # Bottleneck
    b = conv_block(p4, 1024)

    # Decoder with dense skip connections
    d4 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(b)
    d4 = layers.concatenate([d4, e4])
    d4 = conv_block(d4, 512)

    d3 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(d4)
    d3 = layers.concatenate([d3, e3])
    d3 = conv_block(d3, 256)

    d2 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(d3)
    d2 = layers.concatenate([d2, e2])
    d2 = conv_block(d2, 128)

    d1 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(d2)
    d1 = layers.concatenate([d1, e1])
    d1 = conv_block(d1, 64)

    # Output
    outputs = layers.Conv2D(num_classes, (1, 1), activation='sigmoid')(d1)

    model = Model(inputs, outputs)
    return model

# Define the Attention U-Net model
def attention_gate(x, g, inter_channels):
    theta_x = layers.Conv2D(inter_channels, (1, 1), padding='same')(x)
    phi_g = layers.Conv2D(inter_channels, (1, 1), padding='same')(g)

    f = layers.Activation('relu')(layers.add([theta_x, phi_g]))
    psi_f = layers.Conv2D(1, (1, 1), padding='same')(f)
    psi_f = layers.Activation('sigmoid')(psi_f)

    return layers.multiply([x, psi_f])

def attention_unet(input_shape=(128, 128, 3), num_classes=1):
    inputs = layers.Input(input_shape)

    # Encoder
    e1 = conv_block(inputs, 64)
    p1 = layers.MaxPooling2D((2, 2))(e1)

    e2 = conv_block(p1, 128)
    p2 = layers.MaxPooling2D((2, 2))(e2)

    e3 = conv_block(p2, 256)
    p3 = layers.MaxPooling2D((2, 2))(e3)

    e4 = conv_block(p3, 512)
    p4 = layers.MaxPooling2D((2, 2))(e4)

    # Bottleneck
    b = conv_block(p4, 1024)

    # Decoder with attention gates
    g4 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(b)
    att4 = attention_gate(e4, g4, 512)
    d4 = layers.concatenate([g4, att4])
    d4 = conv_block(d4, 512)

    g3 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(d4)
    att3 = attention_gate(e3, g3, 256)
    d3 = layers.concatenate([g3, att3])
    d3 = conv_block(d3, 256)

    g2 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(d3)
    att2 = attention_gate(e2, g2, 128)
    d2 = layers.concatenate([g2, att2])
    d2 = conv_block(d2, 128)

    g1 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(d2)
    att1 = attention_gate(e1, g1, 64)
    d1 = layers.concatenate([g1, att1])
    d1 = conv_block(d1, 64)

    # Output
    outputs = layers.Conv2D(num_classes, (1, 1), activation='sigmoid')(d1)

    model = Model(inputs, outputs)
    return model


In [ ]:
# Instantiate and compile the models
unet_plus = unet_plus_plus(input_shape=(128, 128, 3))
unet_plus.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

attention_model = attention_unet(input_shape=(128, 128, 3))
attention_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train U-Net++
unet_plus.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1, batch_size=32)

# Train Attention U-Net
attention_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1, batch_size=16)

# Evaluate both models
unet_plus_loss, unet_plus_accuracy = unet_plus.evaluate(X_val, y_val)
attention_loss, attention_accuracy = attention_model.evaluate(X_val, y_val)

print(f"U-Net++ Loss: {unet_plus_loss}, Accuracy: {unet_plus_accuracy}")
print(f"Attention U-Net Loss: {attention_loss}, Accuracy: {attention_accuracy}")


 4/24 ━━━━━━━━━━━━━━━━━━━━ 20:35 62s/step - accuracy: 0.4482 - loss: 0.8265

In [ ]:
# Save U-Net++ model
unet_plus.save('unet_plus_model.h5')

# Save Attention U-Net model
attention_model.save('attention_unet_model.h5')

print("Models saved successfully!")


In [ ]:
# fastapi_app.py
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from tensorflow.keras.models import load_model
import numpy as np
import cv2
from PIL import Image
import io

app = FastAPI()

# Load your best performing model (adjust the path as necessary)
model = load_model('unet_plus_model.h5')  # or 'attention_unet_model.h5'

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    # Read the uploaded image file
    contents = await file.read()
    image = Image.open(io.BytesIO(contents))

    # Preprocess the image (resize, normalize)
    image = image.resize((128, 128))  # Resize to match model input size
    image_array = np.array(image) / 255.0  # Normalize to [0, 1]
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension

    # Perform prediction
    prediction = model.predict(image_array)

    # Post-process the prediction
    binary_prediction = (prediction[0] > 0.5).astype(np.uint8) * 255  # Convert to binary mask

    # Convert back to image
    pred_image = Image.fromarray(binary_prediction)

    # Save the prediction image if needed
    pred_image_path = "predicted_mask.png"  # Adjust path as necessary
    pred_image.save(pred_image_path)

    return JSONResponse(content={"message": "Prediction completed!", "predicted_image_path": pred_image_path})

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
